# Event Sampling

## Prerequisites 

To understand how to generate a Model, a MapDataset and how to fit the data, please refer to the `~gammapy.modeling.models.SkyModel` and [simulate_3d](simulate_3d).

## Context 

This tutorial describes how to sample events for an observation of a given Sky model and IRF. The main aim of the tutorial is to explain how to set the minimal configuration for the event sampling and how to obtain an output photon event list.

The core of the simulation lies into the Gammapy `~gammapy.datasets.MapDatasetEventSampler` class, which is based on the inverse cumulative distribution function [(Inverse CDF)](https://en.wikipedia.org/wiki/Cumulative_distribution_function#Inverse_distribution_function_(quantile_function)). 

The `~gammapy.datasets.MapDatasetEventSampler` takes in input a `~gammapy.datasets.Dataset` object containing the Sky model. The class evaluates the map of predicted counts per bin and then samples it, giving in output a set of events with true coordinates, true energies and times of arrival. It is also possible to then apply IRF corrections (i.e. PSF and energy dispersion) in order to obtain reconstructed coordinates and energies of the sampled events. 

The metadata in the event-list are then catched from an `gammapy.data.Observations` object. 

## Objective
To sample events from a given Sky model, obtaining an event-list with coordinates, energies and times for each event.

## Proposed approach

In this section we show how to define an `gammapy.data.Observations` and to create the `~gammapy.datasets.Dataset` object needed for the event sampling (for more info on `~gammapy.datasets.Dataset` objects, please visit this [link](https://docs.gammapy.org/dev/notebooks/analysis_2.html#Preparing-reduced-datasets-geometry)). We define the Sky model and then we sample events from that. We show how to do it for a point-like source and a template map. We finally check consistency by fitting the sampled events with the same Sky model.

## Setup
As usual, we'll start with some general imports...


In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import copy
import astropy.units as u
from astropy.coordinates import SkyCoord
from gammapy.data import DataStore, GTI, Observation
from gammapy.datasets import MapDataset, MapDatasetEventSampler
from gammapy.maps import MapAxis, WcsGeom, Map
from gammapy.irf import load_cta_irfs
from gammapy.makers import MapDatasetMaker
from gammapy.modeling import Fit
from gammapy.modeling.models import (
    Model, Models, SkyModel, 
    PowerLawSpectralModel, 
    PointSpatialModel,
    GaussianSpatialModel,
    SkyDiffuseCube
)
from regions import CircleSkyRegion

### Define an Observation

We firstly create an `gammapy.data.Observations` object that contains the pointing position, the GTIs and the reference IRFs. 

Hereafter, we consider the IRF of the South configuration of the CTA DC1. We set the pointing position of the simulated field at the Galactic Center and we fix the exposure time to 1 hr.

Let's start with some initial settings:

In [ ]:
IRF_FILE = "$GAMMAPY_DATA/cta-1dc/caldb/data/cta/1dc/bcf/South_z20_50h/irf_file.fits"

POINTING = SkyCoord(0.0, 0.0, frame="galactic", unit="deg")
LIVETIME = 1 * u.hr

Now we can create the observation:

In [ ]:
irfs = load_cta_irfs(IRF_FILE)
observation = Observation.create(
    obs_id=1001, pointing=POINTING, livetime=LIVETIME, irfs=irfs
)

### Define the MapDataset

Let's generate the `~gammapy.datasets.Dataset` object: we define the energy axes (true and reconstruncted), the migration axis and the geometry of the observation. 

*This is a crucial point for the correct configuration of the event sampler. Indeed the spatial and energetic binning should be treaten carefully and the finer the better. For this reason, we suggest to define the energy axes by setting a minimum binning of least 10-20 bins per decade for all the sources of interest. The spatial binning may instead be different from source to source and, at first order, it should be adopted a binning significantly smaller than the expected source size.*

For the examples that will be shown hereafter, we set the geometry of the dataset to a field of view of 2degx2deg and we  bin the spatial map with pixels of 0.02 deg.

In [ ]:
ENERGY_AXIS = MapAxis.from_energy_bounds("0.1 TeV", "100 TeV", nbin=10, per_decade=True)
ENERGY_AXIS_TRUE = MapAxis.from_energy_bounds("0.03 TeV", "300 TeV", nbin=20, per_decade=True, name="energy_true")
MIGRA_AXIS = MapAxis.from_bounds(0.5, 2, nbin=150, node_type="edges", name="migra")

WCS_GEOM = WcsGeom.create(
    skydir=POINTING, width=(2, 2), binsz=0.02, frame="galactic", axes=[ENERGY_AXIS]
)

In the following, the dataset is creating by selecting the effective area, background model, the PSF and the Edisp from the IRF. The dataset thus created can be saved into a FITS file just using the `write()` function. We put it into the `evt_sampling` sub-folder:

In [ ]:
%%time
empty = MapDataset.create(WCS_GEOM, energy_axis_true=ENERGY_AXIS_TRUE, migra_axis=MIGRA_AXIS)
maker = MapDatasetMaker(selection=["exposure", "background", "psf", "edisp"])
dataset = maker.run(empty, observation)

dataset.write('./evt_sampling/dataset.fits', overwrite=True)

### Define the Sky model: a point-like source

First of all, let's define a Sky model (see how to create it [here](https://docs.gammapy.org/dev/notebooks/models.html)) for a point-like source centered 0.5 deg far from the Galactic Center and with a power-law spectral shape. We then save the model into a yaml file.

In [ ]:
spectral_model_pwl = PowerLawSpectralModel(index=2,amplitude="1e-12 TeV-1 cm-2 s-1", reference='1 TeV')
spatial_model_point = PointSpatialModel(lon_0= "0 deg", lat_0="0.5 deg", frame='galactic')
sky_model_pntpwl = SkyModel(spectral_model=spectral_model_pwl, spatial_model=spatial_model_point)

models_pntpwl = Models([sky_model_pntpwl])

file_model = "./evt_sampling/point-pwl.yaml"
models_pntpwl.write(file_model, overwrite=True)

### Sampling the source and background events

Now, we can finally add the `~gammapy.modeling.models.SkyModel` we want to simulate to the `~gammapy.datasets.Dataset` container:

In [ ]:
dataset.models.extend(models_pntpwl)
print(dataset.models)

The next step shows how to sample the events with the `~gammapy.datasets.MapDatasetEventSampler` class. The class requests a random number seed generator (that we set with `random_state=0`), the `~gammapy.datasets.Dataset` and the `gammapy.data.Observations` object. From the latter, the `~gammapy.datasets.MapDatasetEventSampler` class takes all the meta data information.

In [ ]:
%%time
sampler = MapDatasetEventSampler(random_state=0)
events = sampler.run(dataset, observation)

The output of the event-sampler is an event list with coordinates, energies and time of arrivals of the source and background events. Source and background events are flagged by the MC_ID identifier (where 0 is the default identifier for the background).

In [ ]:
print(f"Source events: {len(np.where(events.table['MC_ID']==2)[0])}")
print(f"Background events: {len(np.where(events.table['MC_ID']==0)[0])}")

We can inspect the properties of the simulated events as follows:

In [ ]:
events.peek()

Let's write the photon event list to a FITS file:

In [ ]:
events.table.write("./evt_sampling/events_0001.fits", overwrite=True)

You can now load the event list with `Datastore.from_events_files()` and make your own analysis following the instructions in [`analysis_2`](https://docs.gammapy.org/dev/notebooks/analysis_2.html).

#### Generate a skymap
A skymap of the simulated events is the following:

In [ ]:
counts = Map.create(frame="galactic", skydir=(0, 0.), binsz=0.02, npix=(100, 100))
counts.fill_events(events)
counts.plot(add_cbar=True)

#### Fit the simulated data
We can now check the sake of the event sampling by fitting the data (a tutorial of source fitting is [here](https://docs.gammapy.org/dev/notebooks/analysis_2.html#Fit-the-model) and [here](https://docs.gammapy.org/dev/notebooks/simulate_3d)). We make use of the same `~gammapy.modeling.models.SkyModel` adopted for the simulation. 
Hence, we firstly read the `~gammapy.datasets.Dataset` and the model file, and we fill the `~gammapy.datasets.Dataset` with the sampled events.

In [ ]:
dataset = MapDataset.read("./evt_sampling/dataset.fits")
models_sim_point = Models.read("./evt_sampling/point-pwl.yaml")

counts = Map.from_geom(WCS_GEOM)
counts.fill_events(events)
dataset.counts = counts
dataset.models.extend(models_sim_point)

Let's fit the data and look at the results:

In [ ]:
%%time
fit = Fit([dataset])
result = fit.run(optimize_opts={"print_level": 1})
print(result)

In [ ]:
result.parameters.to_table()

The results looks great!

## Extended source using a template
The event sampler can also work with a template `~gammapy.modeling.models.SkyModel`.
Here we use the interstellar emission model map of the Fermi 3FHL, which can found in the GAMMAPY data repository.

We proceed following the same steps showed and we finally have a look at the events properties:

In [ ]:
diffuse = SkyDiffuseCube.read(
    "$GAMMAPY_DATA/fermi-3fhl-gc/gll_iem_v06_gc.fits.gz"
)
models_diffuse = Models([diffuse])

file_model = "./evt_sampling/diffuse.yaml"
models_diffuse.write(file_model, overwrite=True)

In [ ]:
dataset = MapDataset.read("./evt_sampling/dataset.fits")
dataset.models.extend(models_diffuse)
print(dataset.models)

In [ ]:
%%time
sampler = MapDatasetEventSampler(random_state=0)
events = sampler.run(dataset, observation)

In [ ]:
events.peek()

<!-- ## Read simulated event lists with Datastore.from_events_lists
Here we show how to simulate a set of event lists of the same Sky model, but with different GTIs. We make use of the settings we applied previously.
Let's define the GTI firstly, chosing a time start and a duration of the observation: -->

In [ ]:
# tmin = [1, 5, 7] * u.hr
# livetime = [1, 1, 1] * u.hr

<!-- We then simulate the event lists and we save them into a fits file: -->

In [ ]:
# for i in np.arange(len(tmin)):
#     observation = Observation.create(
#         obs_id=i, pointing=POINTING, 
#         tstart=tmin[i], livetime=livetime[i],
#         irfs=irfs
#     )
#     print(observation)

#     dataset = maker.run(empty, observation)
#     dataset.models.extend(models_pntpwl)

#     events = sampler.run(dataset, observation)
#     events.table.write(f"./evt_sampling/events_{i:04d}.fits", overwrite=True)

<!-- The event lists can loaded into a DataStore with the function `from_events_files()`: -->

In [ ]:
# path = Path("./")
# paths = list(path.rglob("events*.fits"))
# data_store = DataStore.from_events_files(paths)
# observations = data_store.get_observations()
# print(data_store)

<!-- The data can be now treated as explained [here](...) -->

## Exercises
- Try to sample events for an extended source (e.g. a radial gaussian morphology);
- Change the spatial model and the spectrum of the simulated Sky model;
- Include a temporal model in the simulation